In [1]:
# imports
import os
import random

import bokeh
from bokeh.plotting import output_notebook
output_notebook() # set default; alternative is output_file()

Loading BokehJS ...

In [2]:
# fix RISE scollbar missing
from traitlets.config.manager import BaseJSONConfigManager
path = "C:\\Users\\chris\\.jupyter\\nbconfig"
cm = BaseJSONConfigManager(config_dir=path)

cm.update("livereveal", {
              "scroll": True,
})

# also check jupyter Cell menu/Toggle Scrolling

{'scroll': True}

# Meaning of Bokeh
- Japanese word **“bokeh”** used in photography to describe *blurring of the out-of-focus parts of an image.*<br><br>
<img src="https://p2.piqsels.com/preview/926/415/630/wheat-ear-dry-harvest.jpg" height=200 style="float:center"> <br><br>
- How do you pronounce this crazy word?
  - bouquet
  - bok-ah
  - **both are fine**

# Bokeh
- [bokeh homepage](https://bokeh.pydata.org/en/latest/index.html)<br>
- python library 
- focused on interactive visualization
- targets browsers for presentation <br>
- goals: elegant, concise, designed for large/streaming data


# Two modes
- client based
- server

# 
- Bokeh uses Python to create high level objects (plots, subplots, lines, etc).
- BokehJS then converts this to JSON format
- ...and then renders everything in Javascript
- for the browser to display

data --> python --> Bokeh --> Bokeh ColumnDataSource--> BokehJS --> Javascript -->Browser --> your eyes


# Bokeh Code

# five ways to interact using Bokeh
- Configuring the toolbar
- Selecting data points
- Adding hover actions
- Linking axes and selections
- Highlighting data using the legend

# also can interact with ipywidgets

In [3]:
from bokeh.plotting import figure, output_notebook, show

x = [x for x in range(0, 11)]
y = [9, 8, 7, 5, 4, 6, 8, 3, 2, 0, 1]

chart = figure(title="simple line chart", 
               x_axis_label="x axis!", y_axis_label="y_axis",
               toolbar_location="right")

chart.xaxis.axis_label_text_font_size = "18pt"
chart.yaxis.axis_label_text_font_size = "18pt"

chart.line(x, y, line_width=2)

show(chart)

# review interactive tools

In [4]:
# adding annotations

from bokeh.models import LassoSelectTool
from bokeh.models import BoxAnnotation

# chart.add_tools(LassoSelectTool())
low_box = BoxAnnotation(top=4.2, bottom=1.6, fill_alpha=0.1, fill_color='green')
chart.add_layout(low_box)

show(chart)

In [5]:
# simple interactivity example - using ipywidgets
# source:  https://github.com/bokeh/bokeh/blob/1.3.4/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb

from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

# define evenly spaced data to plot trig functions
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

p = figure(title="example - simple trg plot", plot_height=300, plot_width=900, y_range=(-10,10),
           background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

def update(function, freq=1, Amplitude=2, phi=0):
    if   function == "sin": func = np.sin
    elif function == "cos": func = np.cos
    elif function == "tan": func = np.tan
    r.data_source.data['y'] = Amplitude * func(freq * x + phi)
    push_notebook()
    

Loading BokehJS ...

In [6]:
show(p, notebook_handle=True)  # handle updates existing plot, only needed in jupyter notebook (not jupyterlab)

interact(update, function=["sin", "cos", "tan"], freq=(0,50), Amplitude=(1,10), phi=(0, 20, 0.1))


interactive(children=(Dropdown(description='function', options=('sin', 'cos', 'tan'), value='sin'), IntSlider(…

<function __main__.update(function, freq=1, Amplitude=2, phi=0)>

# bokeh resources

[bokeh cheat sheet](https://datacamp-community-prod.s3.amazonaws.com/f9511cf4-abb9-4f52-9663-ea93b29ee4b7)

In [7]:
# how to choose visualization

# gapminder chart

In [8]:
# save data from chart to object or file

%matplotlib inline
import numpy as np
from random import choice
from string import ascii_lowercase

from bokeh.models.tools import *
from bokeh.plotting import *
from bokeh.models import CustomJS

output_notebook()

# zero out results var for graph re-use:
results = []

TOOLS="pan,wheel_zoom,reset,hover,poly_select,box_select"
p = figure(title = "My chart", tools=TOOLS)
p.xaxis.axis_label = 'X'
p.yaxis.axis_label = 'Y'

source = ColumnDataSource(
    data=dict(
        xvals=list(range(0, 10)),
        yvals=list(np.random.normal(0, 1, 10)),
        letters = [choice(ascii_lowercase) for _ in range(10)]
    )
)
p.scatter("xvals", "yvals",source=source,fill_alpha=0.2, size=15)

select_tool = p.select(dict(type=BoxSelectTool))[0]

source.callback = CustomJS(args=dict(p=p), code="""
        var inds = cb_obj.get('selected')['1d'].indices;
        var d1 = cb_obj.get('data');
        console.log(d1)
        var kernel = IPython.notebook.kernel;
         IPython.notebook.kernel.execute("inds = " + inds);
        """
)

print(source.data)
print(source.callback)
show(p)

Loading BokehJS ...

{'xvals': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'yvals': [-0.276121912563779, -0.260352425450977, -0.8224472156680797, -1.0509032679303059, 0.05386377116600327, 1.7681900227153804, -0.14545616368058387, 2.0840192292277115, 1.8628728288820284, 0.06807305880633133], 'letters': ['o', 's', 'x', 'w', 'i', 'y', 'n', 'f', 'e', 'b']}
CustomJS(id='1311', ...)


In [9]:
# save data from chart to object or file

%matplotlib inline
import numpy as np
from random import choice
from string import ascii_lowercase

from bokeh.models.tools import *
from bokeh.plotting import *
from bokeh.models import CustomJS

output_notebook()

# zero out results var for graph re-use:
results = []

TOOLS="pan,wheel_zoom,reset,hover,poly_select,box_select"
p = figure(title = "My chart", tools=TOOLS)
p.xaxis.axis_label = 'X'
p.yaxis.axis_label = 'Y'

source1 = ColumnDataSource(
    data=dict(
        xvals=list(range(0, 10)),
        yvals=list(np.random.normal(0, 1, 10)),
        letters = [choice(ascii_lowercase) for _ in range(10)]
    )
)

s2 = ColumnDataSource(data=dict(x=[], y=[]))

p.scatter("xvals", "yvals",source=source1,fill_alpha=0.2, size=15)

select_tool = p.select(dict(type=BoxSelectTool))[0]


source1.selected.js_on_change('indices', CustomJS(args=dict(s1=source1, s2=s2), code="""
        var inds = cb_obj.indices;
        var d1 = s1.data;
        var d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (var i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)
)

show(p)

Loading BokehJS ...

In [10]:

print(source1.selected)
# print(s1.data)
print(s2.data)

Selection(id='1440', ...)
{'x': [], 'y': []}


In [11]:
print(results)
print(source.data['xvals'][0])
print(source.data['yvals'][0])

[]
0
-0.276121912563779


In [12]:
print(type(source.callback))

<class 'bokeh.models.callbacks.CustomJS'>


In [13]:
# zip results in into a list of tuples for x,y

# this is getting all values in plot; not just the ones selected

results = zip([source.data['xvals'][i] for i in range(0, int(len(source.data['xvals']))-1)],
    [source.data['yvals'][i] for i in range(0, int(len(source.data['yvals']))-1)])

print([result for result in results])

[(0, -0.276121912563779), (1, -0.260352425450977), (2, -0.8224472156680797), (3, -1.0509032679303059), (4, 0.05386377116600327), (5, 1.7681900227153804), (6, -0.14545616368058387), (7, 2.0840192292277115), (8, 1.8628728288820284)]


In [14]:
print(source.data['xvals'])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [15]:
source.data['yvals']

[-0.276121912563779,
 -0.260352425450977,
 -0.8224472156680797,
 -1.0509032679303059,
 0.05386377116600327,
 1.7681900227153804,
 -0.14545616368058387,
 2.0840192292277115,
 1.8628728288820284,
 0.06807305880633133]

In [16]:
# bokeh CustomJS callback
# https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html


In [31]:
from random import random

from bokeh.layouts import row, column
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show

# output_file("callback.html")

x = [random() for x in range(500)]
y = [random() for y in range(500)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
s2 = ColumnDataSource(data=dict(x=[], y=[]))
s3 = ColumnDataSource(data=dict(x=[], y=[]))
p1 = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.4)


p2 = figure(plot_width=400, plot_height=400, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")
p2.circle('x', 'y', source=s2, alpha=0.6)

p3 = figure(plot_width=400, plot_height=400, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")
p3.circle('x', 'y', source=s2, alpha=0.9)


s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2), code="""
        var inds = cb_obj.indices;
        var d1 = s1.data;
        var d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (var i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)
)

layout = column(row(p1, p2), p3)


show(layout)

In [24]:
s2

ColumnDataSource(id='1577', ...)

In [23]:
s1

ColumnDataSource(id='1547', ...)

In [20]:
print(s2.data["x"])

[]


In [21]:
s3 = s2
for element in s3:
    print(element)

TypeError: 'ColumnDataSource' object is not iterable

In [ ]:
def get_values():
    global s2
    print(s2.data)

get_values()